In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import json
import pickle as pkl

from absl import app

import time

# Collect Optima via Local Search

## Results from the server: 15 seeds of 100 Random-Restart LS 

In [2]:
import glob2
import json
import numpy as np

result_files = glob2.glob('../Card_optima/new_encoding_200/seed_*_points.txt')

result_files

['../Card_optima/new_encoding_200/seed_2_200_points.txt',
 '../Card_optima/new_encoding_200/seed_7_200_points.txt',
 '../Card_optima/new_encoding_200/seed_9_200_points.txt',
 '../Card_optima/new_encoding_200/seed_12_200_points.txt',
 '../Card_optima/new_encoding_200/seed_3_200_points.txt',
 '../Card_optima/new_encoding_200/seed_8_200_points.txt',
 '../Card_optima/new_encoding_200/seed_15_200_points.txt',
 '../Card_optima/new_encoding_200/seed_4_200_points.txt',
 '../Card_optima/new_encoding_200/seed_6_200_points.txt',
 '../Card_optima/new_encoding_200/seed_11_200_points.txt',
 '../Card_optima/new_encoding_200/seed_5_200_points.txt',
 '../Card_optima/new_encoding_200/seed_10_200_points.txt',
 '../Card_optima/new_encoding_200/seed_1_200_points.txt',
 '../Card_optima/new_encoding_200/seed_14_200_points.txt',
 '../Card_optima/new_encoding_200/seed_0_200_points.txt',
 '../Card_optima/new_encoding_200/seed_13_200_points.txt']

In [3]:
results_per_seed = list()

for file_i in result_files:
    with open(file_i, 'rb') as filehandle:
        # read the data as bina|ry data stream
        res_i = json.load(filehandle)
        results_per_seed.append(res_i)

In [4]:
def card_optima(K_until_repeat, P_D=0.5):

    N_numer = K_until_repeat ** 2
    N_denom = -2 * np.log(1 - P_D) 
    return int(N_numer/ N_denom)

assert card_optima(22) == 349

In [5]:
import pyreadr
import pandas as pd
result = pyreadr.read_r('../nasbench/experiments/nasbench_full_landscape.RData')
result = pd.Series(result)

nasbench_res =  result['land.df']

In [6]:
N_runs = 0
if '100' in result_files[0]:
    N_runs = 100
elif '200' in result_files[0]:
    N_runs = 200
else: 
    pass

In [7]:
result_files

['../Card_optima/new_encoding_200/seed_2_200_points.txt',
 '../Card_optima/new_encoding_200/seed_7_200_points.txt',
 '../Card_optima/new_encoding_200/seed_9_200_points.txt',
 '../Card_optima/new_encoding_200/seed_12_200_points.txt',
 '../Card_optima/new_encoding_200/seed_3_200_points.txt',
 '../Card_optima/new_encoding_200/seed_8_200_points.txt',
 '../Card_optima/new_encoding_200/seed_15_200_points.txt',
 '../Card_optima/new_encoding_200/seed_4_200_points.txt',
 '../Card_optima/new_encoding_200/seed_6_200_points.txt',
 '../Card_optima/new_encoding_200/seed_11_200_points.txt',
 '../Card_optima/new_encoding_200/seed_5_200_points.txt',
 '../Card_optima/new_encoding_200/seed_10_200_points.txt',
 '../Card_optima/new_encoding_200/seed_1_200_points.txt',
 '../Card_optima/new_encoding_200/seed_14_200_points.txt',
 '../Card_optima/new_encoding_200/seed_0_200_points.txt',
 '../Card_optima/new_encoding_200/seed_13_200_points.txt']

In [8]:
N_runs

200

In [9]:
offset_zero_indexing = 1

avg_first_index = list()


for r_i in results_per_seed:
    seed_i, duration_i, stats = r_i
    
    list_ending_points = [s_[0] for s_ in stats]
    list_starting_points = [s_[3] for s_ in stats]
    
    list_steps = [s_[2] + offset_zero_indexing  for s_ in stats]    
    mean_steps = np.mean(list_steps)
    std_steps = np.std(list_steps)
    
    list_acc_test_end = [s_[1] for s_ in stats]
    mean_acc_test_end = np.mean(list_acc_test_end)
    
    list_starting_acc_test = [nasbench_res.loc[s_]['test_acc.36'] for s_ in list_starting_points]
    mean_acc_test_start = np.mean(list_starting_acc_test)
    
    assert len(list_starting_points) == len(list_acc_test_end)
    len_item = len(list_starting_points)
    
    list_improvement = [list_acc_test_end[item_j] - list_starting_acc_test[item_j] for item_j in range(len_item)]
    mean_acc_test_diff = np.mean(list_improvement)
    mean_acc_test_diff = mean_acc_test_diff * 100
    
    #print(list_starting_acc_test)
    distinct_starting = len(set(list_starting_points))
    distinct_ending = len(set(list_ending_points))
    #print(distinct_starting, distinct_ending)
    
    all_models = list()
    idx_first_repeat = 0 #None
    for i, e in enumerate(list_ending_points):
        if e in all_models:
            idx_first_repeat = i + offset_zero_indexing
            break;
        else: 
            all_models.append(e)
    
    avg_first_index.append(idx_first_repeat)
    print("Seed i: ",seed_i, ", Mean step:", mean_steps, 
          ", Mean test accuracy diff: ", mean_acc_test_diff, 
          "", N_runs - distinct_ending, 
          ", Idx of collision: ",idx_first_repeat,'\n')

# Here 0 means Not applicable
print("Average index of first collision:" , avg_first_index, '\n')

Seed i:  2 , Mean step: 2.895 , Mean test accuracy diff:  4.532318425675234  1 , Idx of collision:  94 

Seed i:  7 , Mean step: 2.78 , Mean test accuracy diff:  4.886284751196703  0 , Idx of collision:  0 

Seed i:  9 , Mean step: 2.895 , Mean test accuracy diff:  4.472756455341975  0 , Idx of collision:  0 

Seed i:  12 , Mean step: 2.89 , Mean test accuracy diff:  5.47502663731575  2 , Idx of collision:  57 

Seed i:  3 , Mean step: 2.85 , Mean test accuracy diff:  4.864533425619205  0 , Idx of collision:  0 

Seed i:  8 , Mean step: 2.775 , Mean test accuracy diff:  4.762670166790485  2 , Idx of collision:  26 

Seed i:  15 , Mean step: 3.09 , Mean test accuracy diff:  6.558326671520868  0 , Idx of collision:  0 

Seed i:  4 , Mean step: 2.965 , Mean test accuracy diff:  5.05174941321214  0 , Idx of collision:  0 

Seed i:  6 , Mean step: 3.02 , Mean test accuracy diff:  5.546290752788385  1 , Idx of collision:  58 

Seed i:  11 , Mean step: 3.03 , Mean test accuracy diff:  4.68184

In [10]:
only_pos_rank_occ = list()
for item in avg_first_index:
    if item > 0:
        only_pos_rank_occ.append(item)
only_pos_rank_occ, len(only_pos_rank_occ)

([94, 57, 26, 58, 38, 195, 52, 129, 197], 9)

In [11]:
k_mean = np.mean(only_pos_rank_occ)
est_card_optima = card_optima(k_mean)

print('Average index of first collision over all exp: ', k_mean)
print('Estimate cardial of Local Optima (maxima): '  , est_card_optima)

Average index of first collision over all exp:  94.0
Estimate cardial of Local Optima (maxima):  6373


In [12]:
ratio_of_search_space = est_card_optima / nasbench_res.shape[0] * 100

print('Percentage of size of search space:', ratio_of_search_space )

Percentage of size of search space: 1.504400128415765


In [13]:
mean_step = np.mean([2.895, 2.89, 2.775, 3.02, 2.86, 3.005, 2.825, 2.795, 2.695])
print("Overall mean Step: ", mean_step)

Overall mean Step:  2.862222222222222


In [14]:
avg_improvement = np.mean(
[4.532318425675234,
5.47502663731575,
4.762670166790485,
5.546290752788385,
5.060680480053028, 
4.570262419680755, 
5.774739523728689,
4.779964582373699,
4.786758835117022]
)

print("Overall improvement in accuracy for BILS: ", avg_improvement)

Overall improvement in accuracy for BILS:  5.032079091502561
